In [1]:
# Welcome to your new notebook
# Type here in the cell editor to add code!
import pandas as pd
from pyspark.sql import functions as F
from pyspark.sql.types import *

StatementMeta(, 72407d99-9d7e-4e60-86f7-d0a22b624cb9, 3, Finished, Available, Finished)

In [2]:
df = spark.sql("SELECT * FROM NHL_Lakehouse_Bronze.nhl_game LIMIT 1000")
display(df)
df = (
    df
    .withColumn("home_team_id", F.col("home_team_id").cast("string"))
    .withColumn("away_team_id", F.col("away_team_id").cast("string"))
    .withColumn("game_id", F.col("game_id").cast("string"))
)

StatementMeta(, 72407d99-9d7e-4e60-86f7-d0a22b624cb9, 4, Finished, Available, Finished)

SynapseWidget(Synapse.DataFrame, 0dfd3b9a-6374-4a32-9c62-0d26b7a9f391)

In [3]:
df = spark.sql("SELECT * FROM NHL_Lakehouse_Bronze.nhl_game LIMIT 1000")
display(df)

StatementMeta(, 72407d99-9d7e-4e60-86f7-d0a22b624cb9, 5, Finished, Available, Finished)

SynapseWidget(Synapse.DataFrame, 7559194f-3f22-4820-bff1-7980e11be8e3)

In [5]:
df_game = spark.read.table("nhl_game")
df_team_stats = spark.read.table("nhl_game_teams_stats")
df_teams = spark.read.table("nhl_team_info")

df_game.count()

StatementMeta(, 72407d99-9d7e-4e60-86f7-d0a22b624cb9, 6, Finished, Available, Finished)

26305

In [6]:
home_stats = (
    df_team_stats
    .withColumnRenamed("team_id", "home_team_id")
    .withColumnRenamed("goals", "home_goals")
    .select("game_id", "home_team_id", "home_goals")
)

away_stats = (
    df_team_stats
    .withColumnRenamed("team_id", "away_team_id")
    .withColumnRenamed("goals", "away_goals")
    .select("game_id", "away_team_id", "away_goals")
)


StatementMeta(, 72407d99-9d7e-4e60-86f7-d0a22b624cb9, 7, Finished, Available, Finished)

In [7]:
df_joined = (
    df_game
    .join(home_stats, "game_id", "left")
    .join(away_stats, "game_id", "left")
)

#df_joined.count()


StatementMeta(, 72407d99-9d7e-4e60-86f7-d0a22b624cb9, 8, Finished, Available, Finished)

In [8]:
df_joined.show(5)

StatementMeta(, 72407d99-9d7e-4e60-86f7-d0a22b624cb9, 9, Finished, Available, Finished)

+----------+--------+----+-------------------+------------+------------+----------+----------+------------+--------------------+--------------+-------------------+------------------+----------------------+------------------+------------+----------+------------+----------+
|   game_id|  season|type|      date_time_gmt|away_team_id|home_team_id|away_goals|home_goals|     outcome|home_rink_side_start|         venue|         venue_link|venue_time_zone_id|venue_time_zone_offset|venue_time_zone_tz|home_team_id|home_goals|away_team_id|away_goals|
+----------+--------+----+-------------------+------------+------------+----------+----------+------------+--------------------+--------------+-------------------+------------------+----------------------+------------------+------------+----------+------------+----------+
|2016020045|20162017|   R|2016-10-19 00:30:00|           4|          16|         4|         7|home win REG|               right| United Center|/api/v1/venues/null|   America/Chicago

In [9]:
from pyspark.sql import functions as F

g = df_game.alias("g")
h = home_stats.alias("h")
a = away_stats.alias("a")

df_joined = g.join(h, "game_id", "left").join(a, "game_id", "left")

StatementMeta(, 72407d99-9d7e-4e60-86f7-d0a22b624cb9, 10, Finished, Available, Finished)

In [10]:
df_final = (
    df_joined
    .withColumn("total_goals", F.col("g.home_goals") + F.col("g.away_goals"))
    .withColumn("home_win_flag", F.col("g.home_goals") > F.col("g.away_goals"))
    .withColumnRenamed("date_time_gmt", "game_date")
    .select(
        F.col("g.game_id").cast("string"),
        F.col("g.season"),
        F.col("game_date"),
        F.col("g.home_team_id").cast("string"),
        F.col("g.away_team_id").cast("string"),
        F.col("g.home_goals"),
        F.col("g.away_goals"),
        F.col("total_goals"),
        F.col("home_win_flag"),
        F.col("g.venue")
        # Optional: F.col("g.game_type") if needed
    )
)

df_final.count()

StatementMeta(, 72407d99-9d7e-4e60-86f7-d0a22b624cb9, 11, Finished, Available, Finished)

166900

In [11]:
df_final.head(5)

StatementMeta(, 72407d99-9d7e-4e60-86f7-d0a22b624cb9, 12, Finished, Available, Finished)

[Row(game_id='2016020045', season=20162017, game_date=datetime.datetime(2016, 10, 19, 0, 30), home_team_id='16', away_team_id='4', home_goals=7, away_goals=4, total_goals=11, home_win_flag=True, venue='United Center'),
 Row(game_id='2016020045', season=20162017, game_date=datetime.datetime(2016, 10, 19, 0, 30), home_team_id='16', away_team_id='4', home_goals=7, away_goals=4, total_goals=11, home_win_flag=True, venue='United Center'),
 Row(game_id='2016020045', season=20162017, game_date=datetime.datetime(2016, 10, 19, 0, 30), home_team_id='16', away_team_id='4', home_goals=7, away_goals=4, total_goals=11, home_win_flag=True, venue='United Center'),
 Row(game_id='2016020045', season=20162017, game_date=datetime.datetime(2016, 10, 19, 0, 30), home_team_id='16', away_team_id='4', home_goals=7, away_goals=4, total_goals=11, home_win_flag=True, venue='United Center'),
 Row(game_id='2017020812', season=20172018, game_date=datetime.datetime(2018, 2, 7, 0, 0), home_team_id='7', away_team_id='2

In [12]:
duplicates_df = (
    df_final
    .groupBy("game_id","game_date")
    .count()
    .filter(F.col("count") > 1)
)

duplicates_df.count()


StatementMeta(, 72407d99-9d7e-4e60-86f7-d0a22b624cb9, 13, Finished, Available, Finished)

23735

In [13]:
valid_teams = df_teams.select("team_id").distinct()

invalid_home = (
    df_final
    .join(valid_teams, df_final.home_team_id == valid_teams.team_id, "left_anti")
    .withColumn("issue", F.lit("INVALID_HOME_TEAM"))
)

invalid_away = (
    df_final
    .join(valid_teams, df_final.away_team_id == valid_teams.team_id, "left_anti")
    .withColumn("issue", F.lit("INVALID_AWAY_TEAM"))
)

invalid_teams_df = invalid_home.unionByName(invalid_away)

invalid_teams_df.count()

StatementMeta(, 72407d99-9d7e-4e60-86f7-d0a22b624cb9, 14, Finished, Available, Finished)

320

In [14]:
duplicates_df.write.mode("overwrite") \
    .option("header", True) \
    .csv("Files/quarantine/duplicate_games")

invalid_teams_df.write.mode("overwrite") \
    .option("header", True) \
    .csv("Files/quarantine/invalid_team_references")


StatementMeta(, 72407d99-9d7e-4e60-86f7-d0a22b624cb9, 15, Finished, Available, Finished)

In [15]:
df_final.count()

StatementMeta(, 72407d99-9d7e-4e60-86f7-d0a22b624cb9, 16, Finished, Available, Finished)

166900

In [16]:
from pyspark.sql import Window
from pyspark.sql import functions as F

# Define a window partitioned by the columns that define duplicates
window_spec = Window.partitionBy("game_id", "game_date").orderBy(F.lit(1))  # order doesn't matter if you just want first

# Add row_number for each group
df_with_row = df_final.withColumn("row_num", F.row_number().over(window_spec))

# Keep only the first row of each group
clean_df = df_with_row.filter(F.col("row_num") == 1).drop("row_num")

clean_df.head(5)

StatementMeta(, 72407d99-9d7e-4e60-86f7-d0a22b624cb9, 17, Finished, Available, Finished)

[Row(game_id='2000020001', season=20002001, game_date=datetime.datetime(2000, 10, 4, 23, 0), home_team_id='25', away_team_id='21', home_goals=2, away_goals=2, total_goals=4, home_win_flag=False, venue='Reunion Arena'),
 Row(game_id='2000020002', season=20002001, game_date=datetime.datetime(2000, 10, 5, 23, 0), home_team_id='6', away_team_id='9', home_goals=4, away_goals=4, total_goals=8, home_win_flag=False, venue='FleetCenter'),
 Row(game_id='2000020003', season=20002001, game_date=datetime.datetime(2000, 10, 5, 23, 0), home_team_id='7', away_team_id='16', home_goals=4, away_goals=2, total_goals=6, home_win_flag=True, venue='HSBC Arena'),
 Row(game_id='2000020004', season=20002001, game_date=datetime.datetime(2000, 10, 5, 23, 0), home_team_id='4', away_team_id='23', home_goals=6, away_goals=3, total_goals=9, home_win_flag=True, venue='First Union Center'),
 Row(game_id='2000020005', season=20002001, game_date=datetime.datetime(2000, 10, 5, 23, 0), home_team_id='20', away_team_id='17',

In [17]:
clean_df.cache()

clean_df.count()  # triggers caching

clean_df.write \
    .format("delta") \
    .mode("overwrite") \
     .option("overwriteSchema", "true") \
    .saveAsTable("NHL_Lakehouse_Silver.silver_game") 

clean_df.write \
    .format("delta") \
    .mode("overwrite") \
    .option("overwriteSchema", "true") \
    .saveAsTable("NHL_Lakehouse_Silver.silver_game")

location = spark.sql("DESCRIBE DETAIL NHL_Lakehouse_Silver.silver_game").collect()[0]['location']
print(location)

StatementMeta(, 72407d99-9d7e-4e60-86f7-d0a22b624cb9, 18, Finished, Available, Finished)

abfss://9fa45eaf-a701-46a3-ba5c-c97dab18050d@onelake.dfs.fabric.microsoft.com/b02d8b6f-ce55-4598-b5a6-52d4209a76fa/Tables/silver_game


In [18]:
spark.table("NHL_Lakehouse_Silver.silver_game").count()

StatementMeta(, 72407d99-9d7e-4e60-86f7-d0a22b624cb9, 19, Finished, Available, Finished)

23735

In [19]:
clean_df = df_final \
    .join(duplicates_df.select("game_id"), "game_id", "left_anti") \
    .join(invalid_teams_df.select("game_id"), "game_id", "left_anti")

StatementMeta(, 72407d99-9d7e-4e60-86f7-d0a22b624cb9, 20, Finished, Available, Finished)

In [20]:
df = spark.sql("SELECT * FROM NHL_Lakehouse_Silver.silver_game")

null_counts = df.select([F.count(F.when(F.col(c).isNull(), c)).alias(c) for c in df.columns])

null_counts.show()



StatementMeta(, 72407d99-9d7e-4e60-86f7-d0a22b624cb9, 21, Finished, Available, Finished)

+-------+------+---------+------------+------------+----------+----------+-----------+-------------+-----+
|game_id|season|game_date|home_team_id|away_team_id|home_goals|away_goals|total_goals|home_win_flag|venue|
+-------+------+---------+------------+------------+----------+----------+-----------+-------------+-----+
|      0|     0|        0|           0|           0|         0|         0|          0|            0|    0|
+-------+------+---------+------------+------------+----------+----------+-----------+-------------+-----+



In [21]:
df.select([F.count(F.when(F.col(c).isNull() | (F.col(c) == ""), c)).alias(c) for c in df.columns]).show()

StatementMeta(, 72407d99-9d7e-4e60-86f7-d0a22b624cb9, 22, Finished, Available, Finished)

+-------+------+---------+------------+------------+----------+----------+-----------+-------------+-----+
|game_id|season|game_date|home_team_id|away_team_id|home_goals|away_goals|total_goals|home_win_flag|venue|
+-------+------+---------+------------+------------+----------+----------+-----------+-------------+-----+
|      0|     0|        0|           0|           0|         0|         0|          0|            0|    0|
+-------+------+---------+------------+------------+----------+----------+-----------+-------------+-----+



In [22]:
df = spark.sql("SELECT * FROM NHL_Lakehouse_Silver.silver_shift LIMIT 1000")
display(df)

StatementMeta(, 72407d99-9d7e-4e60-86f7-d0a22b624cb9, 23, Finished, Available, Finished)

SynapseWidget(Synapse.DataFrame, da0bc3ea-1893-49c0-8122-36d1610caca0)

In [23]:
df = spark.sql("SELECT * FROM NHL_Lakehouse_Silver.silver_game LIMIT 1000")
display(df)

StatementMeta(, 72407d99-9d7e-4e60-86f7-d0a22b624cb9, 24, Finished, Available, Finished)

SynapseWidget(Synapse.DataFrame, eb41d2c5-4380-4a41-92ed-8777ea3925b0)

In [24]:
"""
g = df_game.alias("g")
h = home_stats.alias("h")
a = away_stats.alias("a")
 

df_final = df_joined.select(
    F.col("nhl_game.game_id").alias("game_id"),
    F.col("nhl_game.season").alias("season"),
    F.col("nhl_game.date_time_gmt").alias("game_date"),
    F.col("nhl_game.home_team_id").alias("home_team_id"),
    F.col("nhl_game.away_team_id").alias("away_team_id"),
    F.col("nhl_game.home_goals").alias("home_goals"),
    F.col("nhl_game.away_goals").alias("away_goals"),
    (
        F.coalesce(F.col("nhl_game.home_goals"), F.lit(0)) +
        F.coalesce(F.col("nhl_game.away_goals"), F.lit(0))
    ).alias("total_goals"),
    (
        F.col("nhl_game.home_goals") > F.col("nhl_game.away_goals")
    ).alias("home_win_flag"),
    F.col("nhl_game.venue").alias("venue")
)

print(df_final.head())
"""

StatementMeta(, 72407d99-9d7e-4e60-86f7-d0a22b624cb9, 25, Finished, Available, Finished)

'\ng = df_game.alias("g")\nh = home_stats.alias("h")\na = away_stats.alias("a")\n \n\ndf_final = df_joined.select(\n    F.col("nhl_game.game_id").alias("game_id"),\n    F.col("nhl_game.season").alias("season"),\n    F.col("nhl_game.date_time_gmt").alias("game_date"),\n    F.col("nhl_game.home_team_id").alias("home_team_id"),\n    F.col("nhl_game.away_team_id").alias("away_team_id"),\n    F.col("nhl_game.home_goals").alias("home_goals"),\n    F.col("nhl_game.away_goals").alias("away_goals"),\n    (\n        F.coalesce(F.col("nhl_game.home_goals"), F.lit(0)) +\n        F.coalesce(F.col("nhl_game.away_goals"), F.lit(0))\n    ).alias("total_goals"),\n    (\n        F.col("nhl_game.home_goals") > F.col("nhl_game.away_goals")\n    ).alias("home_win_flag"),\n    F.col("nhl_game.venue").alias("venue")\n)\n\nprint(df_final.head())\n'